In [7]:
# -*- coding: utf-8 -*-
import numpy
import codecs
import json
import time
import operator as op

In [36]:
def preprocess():
    f = codecs.open('./data/user_click_data.txt', 'r', 'utf-8')
    f_training = codecs.open('./data/_training.json', 'w', 'utf-8')
    f_validation = codecs.open('./data/_validation.json', 'w', 'utf-8')
    f_user_data_training = codecs.open('./data/_user_data_training.json', 'w', 'utf-8')
    f_user_data_validation = codecs.open('./data/_user_data_validation.json', 'w', 'utf-8')
    f_news_data = codecs.open('./data/_news_data.json', 'w', 'utf-8')
    f_user_data_training_clean = codecs.open('./data/_user_data_training_clean.json', 'w', 'utf-8')
    f_user_data_validation_clean = codecs.open('./data/_user_data_validation_clean.json', 'w', 'utf-8')
    
    i=0
    user_data_training = {}
    news_data = {}
    user_data_validation = {}
    user_data_training_clean = {}
    user_data_validation_clean = {}
    #news_data_validation = {}
    for line in f:
        i+=1
        if(i % 10000 == 20):
            print(i)
        partitions = line.split('\t')
        #user_id, news_id, click_time, title, article, news_time
        user_id = int(partitions[0])
        news_id = int(partitions[1])
        click_time = int(partitions[2])
        if partitions[4] == "NULL":
            #print(partitions[3], tstp)
            continue
        # deal with news_time
        try:
            tstp = transform_time(partitions[5])  
        except:
            if partitions[4] == "NULL":
                continue
#                 print(partitions[4], len(partitions[4]))
            tstp = int(1393603200)
        #data = {"user_id": user_id, "news_id": news_id, "click_time": click_time,
        #       "title": partitions[3], "article": partitions[4], "news_time": tstp}
        day = int((int(partitions[2]) - 1393603200) / 86400) + 1
        if day <= 20:
            # the first 20 days records belong to training set
            #json.dump(data, f_training)
            if user_id not in user_data_training:
                user_data_training.setdefault(user_id, {})
            user_data_training[user_id].setdefault(news_id, click_time)
        else:
            # the last 10 days records belong to validation set
            #json.dump(data, f_validation)
            if user_id not in user_data_validation:
                user_data_validation.setdefault(user_id, {})
            user_data_validation[user_id].setdefault(news_id, click_time)
            
        if news_id in news_data:
            if len(partitions[4]) > len(news_data[news_id][1]):
                # if necessary,
                # update the news info
                print(1)
                news_data[news_id][1] = partitions[4]
                news_data[news_id][2] = tstp
            #if op.eq(partitions[4], news_data[news_id][1]) != 0 and \
            #op.eq(partitions[3], news_data[news_id][0]) != 0 and news_data[news_id][2] != tstp:
    #   print(partitions[3], partitions[4])
        else:
            #[news_title, news_article, news_time]
            news_data.setdefault(news_id, [partitions[3], partitions[4], tstp])
    
    user_validation = user_data_validation.keys()
    user_training = user_data_training.keys()
#     zombie = list(set(user_validation) ^ set(user_training))
#     ret1 = [ i for i in user_validation if i not in user_training ]
#     ret2 = [ i for i in user_training if i not in user_validation ]
#     user_list = [ i for i in user_training if i in user_validation ]
#     print(len(zombie), len(ret1), len(ret2), len(user_validation), len(user_training), len(user_list))
    user_list = [ i for i in user_training if i in user_validation ]
    for i in user_list:
        user_data_validation_clean.setdefault(i, user_data_validation[i])
        user_data_training_clean.setdefault(i, user_data_training[i])
    # dump into files
    json.dump(user_data_training_clean, f_user_data_training_clean)
    json.dump(user_data_validation_clean, f_user_data_validation_clean)
    json.dump(user_data_training, f_user_data_training)
    json.dump(user_data_validation, f_user_data_validation)
    json.dump(news_data, f_news_data)
    # close files
    f_training.close()
    f_validation.close()
    f_user_data_training.close()
    f_user_data_validation.close()
    f_news_data.close()
    f.close()
    
def transform_time(t):
    # t :
    # 2014年03月xx日xx:xx
    tmp = t.split("月")
    tmp = tmp[1].split("日")
    day = int(tmp[0])
    tmp = tmp[1].split(":")
    hour = int(tmp[0])
    minute = int(tmp[1].split("\r")[0])
    # 1393603200 >>> 2014-03-01 00:00:00
    timestamp = 1393603200 + (day - 1) * 86400 + hour * 3600 + minute * 60
    return int(timestamp)

preprocess()

20
楼市踏空风险加剧？ 1394427660
政协发言人回应周永康问题 1393811700
楼继伟：推进房地产税立法 1394078700
冤假错案改变的人生 1394596800
港漂生活微记录（四） 1393819380
港漂生活微记录（五） 1393819440
港漂生活微记录（二） 1393819320
港漂生活微记录（一） 1393819260
财经人物汇（12月21日） 1395532800
该不该为公务员涨工资？ 1394516460
克里米亚何去何从？ 1394112120
财新记者于海荣在央广谈再改国企 1394420400
财新记者刘冉在央广谈新媒体时代的公关行业 1395637080
比特币还值得投资吗？ 1395365940
快评财新闻（12月30日） 1396165860
财新辩论—李克强经济学 1395015120
京东打响O2O全民战争 1395393180
国企改革的冲动与担忧 1394097060
克里米亚将成欧洲新的火药桶？ 1394062980
“余额宝们”还有火多久？ 1393892640
政协委员谈经济改革 1394100840
雾霾模糊了我们的未来？ 1393837140
电影工业发展的未来趋势 1393819800
人民币缘何五连跌？ 1395890460
朱民：诊断世界与中国经济 1395649980
美俄新一轮角力开始？ 1395219360
前瞻习近平访欧之旅 1395039120
【请问】盖茨大预言 1395020760
追忆“娱乐之王”邵逸夫 1394161980
李克强答中外记者问实录 1394691000
马官方更新客机失联地点 1394676060
政协开幕会为昆明遇难者默哀 1393900500
#新春贺喜#【舒立时间】热议房地产泡沫 1395807720
【舒立时间】向世界提问 1394720880
【请问】史景迁：我的中国研究 1395740340
李克强首做政府工作报告 1394004300
10020
余额宝如何破围银行夹击？ 1394677980
财新编辑李涛在央广谈民营银行闯关 1395030300
互联网信用卡被叫停因为“安全”？ 1395114060
利率市场化迈出最后一步 1394787720
“一行三会”负责人答记者问 1394511360
失联客机落入南印度洋 13957144

In [ ]:
a = "2014-03-01 00:00:00"
timeArray = time.strptime(a, "%Y-%m-%d %H:%M:%S")
timeStamp = int(time.mktime(timeArray))
print(timeStamp)

    2014-03-01 00:00:00
    ↓
    1393603200

In [2]:
file = open('./_user_data_training.json','r',encoding='utf-8')
s = json.load(file)
print(s)

{'5218791': {'100648598': 1394463264, '100648802': 1394463205, '100648830': 1394463196, '100648915': 1394463181, '100648984': 1394463158, '100649040': 1394463120}, '52550': {'100644102': 1393607588, '100644648': 1393607490, '100643974': 1393607261}}


In [ ]:
def reclean_news():
    f_news_data = codecs.open('./data/_news_data.json', 'r', 'utf-8')
    news_data = json.load(f_news_data)
    
    
    
    
    
    f_news_data.close()
    